Imports -> Tensorfkow/Keras for LSTM , Bidirectional LSTM

In [0]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Embedding, TimeDistributed
from tensorflow.keras.preprocessing.sequence import pad_sequences
word_to_index=spark.readcsv("abfss://wmu-fm@wmu.dbfs.core.windows.net/train.csv")
label_to_index=spark.readcsv("abfss://wmu-fm@wmu.dbfs.core.windows.net/train.csv")
det train_model():
# Model definition
    vocab_size = len(word_to_index)
    embedding_dim = 50
    hidden_dim = 64
    output_dim = len(label_to_index)

    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(TimeDistributed(Dense(output_dim, activation='softmax')))

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(texts_indices, np.expand_dims(labels_indices, -1), epochs=10, batch_size=32)

    test_text_indices = [word_to_index.get(word, word_to_index["<UNK>"]) for word in test_text.split()]
    padded_test_text_indices = pad_sequences([test_text_indices], maxlen=max_length, padding='post')
    predicted_labels = model.predict_classes(padded_test_text_indices)
    predicted_labels = [index_to_label[label] for label in predicted_labels[0] if label != 0]
    return predicted_labels

##
Converting the raw text data and corresponding labels into numerical indices, which can be fed into the neural network model for training. 

In [0]:
import pandas as pd
from azure.storage.blob import BlobServiceClient
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

def read_csv_from_azure(account_name, account_key, container_name, blob_name):
    connection_string = f"DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net"
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)

    with blob_client as blob:
        data = blob.download_blob()
        df = pd.read_csv(data.content_as_text())
    
    return df

def preprocess_data(df, max_words, max_sequence_length):
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(df['extracted_text'])
    X = tokenizer.texts_to_sequences(df['extracted_text'])
    X = pad_sequences(X, maxlen=max_sequence_length)
    y = df['label']

    return X, y, tokenizer.word_index

def train_model(X, y, vocab_size, max_sequence_length):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size + 1, 64, input_length=max_sequence_length),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X, y, epochs=10, validation_split=0.2, batch_size=32)

    return model

def train_model_from_azure_storage(account_name, account_key, container_name, blob_name, max_words=10000, max_sequence_length=100):
    # Read data from Azure storage
    df = read_csv_from_azure(account_name, account_key, container_name, blob_name)

    # Preprocess data
    X, y, word_index = preprocess_data(df, max_words, max_sequence_length)
    vocab_size = len(word_index)

    # Train the model
    model = train_model(X, y, vocab_size, max_sequence_length)

    account_name = 'wmu'
    account_key = dbutils.secrets.get(scope='wmu-scope',key='wmu-fm-storage-key')
    container_name = 'wmu-fm'
    blob_name = 'train.csv'

    trained_model = train_model_from_azure_storage(account_name, account_key, container_name, blob_name)
    for idx, invoice in enumerate(invoices.documents):
                print("--------Recognizing invoice #{}--------".format(idx + 1))
                vendor_name = invoice.fields.get("VendorName")
                if vendor_name:
                    print(
                        "Vendor Name: {} has confidence: {}".format(
                            vendor_name.value, vendor_name.confidence
                        )
                    )
                vendor_address = invoice.fields.get("VendorAddress")
                if vendor_address:
                    print(
                        "Vendor Address: {} has confidence: {}".format(
                            vendor_address.value, vendor_address.confidence
                        )
                    )
                vendor_address_recipient = invoice.fields.get("VendorAddressRecipient")
                if vendor_address_recipient:
                    print(
                        "Vendor Address Recipient: {} has confidence: {}".format(
                            vendor_address_recipient.value, vendor_address_recipient.confidence
                        )
                    )
                customer_name = invoice.fields.get("CustomerName")
                if customer_name:
                    print(
                        "Customer Name: {} has confidence: {}".format(
                            customer_name.value, customer_name.confidence
                        )
                    )
                customer_id = invoice.fields.get("CustomerId")
                if customer_id:
                    print(
                        "Customer Id: {} has confidence: {}".format(
                            customer_id.value, customer_id.confidence
                        )
                    )
                customer_address = invoice.fields.get("CustomerAddress")
                if customer_address:
                    print(
                        "Customer Address: {} has confidence: {}".format(
                            customer_address.value, customer_address.confidence
                        )
                    )
                customer_address_recipient = invoice.fields.get("CustomerAddressRecipient")
                if customer_address_recipient:
                    print(
                        "Customer Address Recipient: {} has confidence: {}".format(
                            customer_address_recipient.value,
                            customer_address_recipient.confidence,
                        )
                    )
                invoice_id = invoice.fields.get("InvoiceId")
                if invoice_id:
                    print(
                        "Invoice Id: {} has confidence: {}".format(
                            invoice_id.value, invoice_id.confidence
                        )
                    )
                invoice_date = invoice.fields.get("InvoiceDate")
                if invoice_date:
                    print(
                        "Invoice Date: {} has confidence: {}".format(
                            invoice_date.value, invoice_date.confidence
                        )
                    )
                invoice_total = invoice.fields.get("InvoiceTotal")
                if invoice_total:
                    print(
                        "Invoice Total: {} has confidence: {}".format(
                            invoice_total.value, invoice_total.confidence
                        )
                    )
                due_date = invoice.fields.get("DueDate")
                if due_date:
                    print(
                        "Due Date: {} has confidence: {}".format(
                            due_date.value, due_date.confidence
                        )
                    )
                purchase_order = invoice.fields.get("PurchaseOrder")
                if purchase_order:
                    print(
                        "Purchase Order: {} has confidence: {}".format(
                            purchase_order.value, purchase_order.confidence
                        )
                    )
                billing_address = invoice.fields.get("BillingAddress")
                if billing_address:
                    print(
                        "Billing Address: {} has confidence: {}".format(
                            billing_address.value, billing_address.confidence
                        )
                    )
                billing_address_recipient = invoice.fields.get("BillingAddressRecipient")
                if billing_address_recipient:
                    print(
                        "Billing Address Recipient: {} has confidence: {}".format(
                            billing_address_recipient.value,
                            billing_address_recipient.confidence,
                        )
                    )
                shipping_address = invoice.fields.get("ShippingAddress")
                if shipping_address:
                    print(
                        "Shipping Address: {} has confidence: {}".format(
                            shipping_address.value, shipping_address.confidence
                        )
                    )
                shipping_address_recipient = invoice.fields.get("ShippingAddressRecipient")
                if shipping_address_recipient:
                    print(
                        "Shipping Address Recipient: {} has confidence: {}".format(
                            shipping_address_recipient.value,
                            shipping_address_recipient.confidence,
                        )
                    )
                print("Invoice items:")
                for idx, item in enumerate(invoice.fields.get("Items").value):
                    print("...Item #{}".format(idx + 1))
                    item_description = item.value.get("Description")
                    if item_description:
                        print(
                            "......Description: {} has confidence: {}".format(
                                item_description.value, item_description.confidence
                            )
                        )
                    item_quantity = item.value.get("Quantity")
                    if item_quantity:
                        print(
                            "......Quantity: {} has confidence: {}".format(
                                item_quantity.value, item_quantity.confidence
                            )
                        )
                    unit = item.value.get("Unit")
                    if unit:
                        print(
                            "......Unit: {} has confidence: {}".format(
                                unit.value, unit.confidence
                            )
                        )
                    unit_price = item.value.get("UnitPrice")
                    if unit_price:
                        print(
                            "......Unit Price: {} has confidence: {}".format(
                                unit_price.value, unit_price.confidence
                            )
                        )
                    product_code = item.value.get("ProductCode")
                    if product_code:
                        print(
                            "......Product Code: {} has confidence: {}".format(
                                product_code.value, product_code.confidence
                            )
                        )
                    item_date = item.value.get("Date")
                    if item_date:
                        print(
                            "......Date: {} has confidence: {}".format(
                                item_date.value, item_date.confidence
                            )
                        )
                    tax = item.value.get("Tax")
                    if tax:
                        print(
                            "......Tax: {} has confidence: {}".format(tax.value, tax.confidence)
                        )
                    amount = item.value.get("Amount")
                    if amount:
                        print(
                            "......Amount: {} has confidence: {}".format(
                                amount.value, amount.confidence
                            )
                        )
                subtotal = invoice.fields.get("SubTotal")
                if subtotal:
                    print(
                        "Subtotal: {} has confidence: {}".format(
                            subtotal.value, subtotal.confidence
                        )
                    )
                total_tax = invoice.fields.get("TotalTax")
                if total_tax:
                    print(
                        "Total Tax: {} has confidence: {}".format(
                            total_tax.value, total_tax.confidence
                        )
                    )
                previous_unpaid_balance = invoice.fields.get("PreviousUnpaidBalance")
                if previous_unpaid_balance:
                    print(
                        "Previous Unpaid Balance: {} has confidence: {}".format(
                            previous_unpaid_balance.value, previous_unpaid_balance.confidence
                        )
                    )
                amount_due = invoice.fields.get("AmountDue")
                if amount_due:
                    print(
                        "Amount Due: {} has confidence: {}".format(
                            amount_due.value, amount_due.confidence
                        )
                    )
                service_start_date = invoice.fields.get("ServiceStartDate")
                if service_start_date:
                    print(
                        "Service Start Date: {} has confidence: {}".format(
                            service_start_date.value, service_start_date.confidence
                        )
                    )
                service_end_date = invoice.fields.get("ServiceEndDate")
                if service_end_date:
                    print(
                        "Service End Date: {} has confidence: {}".format(
                            service_end_date.value, service_end_date.confidence
                        )
                    )
                service_address = invoice.fields.get("ServiceAddress")
                if service_address:
                    print(
                        "Service Address: {} has confidence: {}".format(
                            service_address.value, service_address.confidence
                        )
                    )
                service_address_recipient = invoice.fields.get("ServiceAddressRecipient")
                if service_address_recipient:
                    print(
                        "Service Address Recipient: {} has confidence: {}".format(
                            service_address_recipient.value,
                            service_address_recipient.confidence,
                        )
                    )
                remittance_address = invoice.fields.get("RemittanceAddress")
                if remittance_address:
                    print(
                        "Remittance Address: {} has confidence: {}".format(
                            remittance_address.value, remittance_address.confidence
                        )
                    )
                remittance_address_recipient = invoice.fields.get("RemittanceAddressRecipient")
                if remittance_address_recipient:
                    print(
                        "Remittance Address Recipient: {} has confidence: {}".format(
                            remittance_address_recipient.value,
                            remittance_address_recipient.confidence,
                        )
                    )
                print("----------------------------------------")


In [0]:
# Pad sequences to the same length
max_length = max(len(seq) for seq in texts_indices)
texts_indices = pad_sequences(texts_indices, maxlen=max_length, padding='post')
labels_indices = pad_sequences(labels_indices, maxlen=max_length, padding='post')

This code defines a recurrent neural network (RNN) model, specifically a Bidirectional LSTM (Long Short-Term Memory) model with a TimeDistributed layer for sequence labeling or sequence classification tasks.



In [0]:
# Model definition
vocab_size = len(word_to_index)
embedding_dim = 50
hidden_dim = 64
output_dim = len(label_to_index)

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
model.add(TimeDistributed(Dense(output_dim, activation='softmax')))
